# Intro
En una empresa dedicada la generación de leads dentro del mercado del Real Estate en USA se necesita saber cuál es el anuncio de Facebook que más conversion rate tiene para cada cliente


La empresa actualmente tiene alrededor de 50 clientes. Las herramientas son un CRM y Facebook Ads.
El CRM permite exportar en CSV la información de los leads, previamente habiendo seleccionado la información que queremos de dichos leads y haberlos filtrado. Para esto, el CRM sólo brinda la posibilidad de seleccionar y filtrar manualmente cuenta por cuenta, por lo que necesitamos herramientas de Web Scrapping para facilitar el armado de la base de datos.
Luego de eso se tiene que unir cada CSV para tener una sólo base de datos que a posterior se debera limpiar para poder realizar el análisis exploratorio.
Ya con la base de datos lista, queda obtener la información necesaria, analizarla y presentarla para poder tomar decisiones en cuanto a la calidad de los Ads



## Código

In [115]:
# importamos todas las librerias necesarios

from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import pandas as pd

from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [117]:
# Preparo el driver

ser=Service('Documents/Programación/Web scrapping/Selenium/chromedriver.exe')
driver = webdriver.Chrome(service=ser)

In [118]:
# Abro la pagina principal del CRM

url = "https://login.CRMNAME.org/"

driver.get(url)
time.sleep(2)


In [119]:
#INICIO SESIÓN CON CODIGO DE SEGURIDAD


#cargo las credenciales
user= input()      #configurar interfaz del input
passw= input()     #configurar interfaz del input    


#ingreso las credenciales en los campos
driver.find_element(by=By.XPATH,value='//INPUT[@name="email"]').send_keys(user)
driver.find_element(by=By.XPATH,value='//INPUT[@name="password"]').send_keys(passw)

driver.find_element(by=By.XPATH,value='//*[@id="app"]/div[3]/section/div[2]/div/div/div/div[4]/div[2]/button').click()
time.sleep(2)

#ya en la siguiente pantalla realizo el envío de 
boton_sendcode = driver.find_element(by=By.XPATH,value='/html/body/div[1]/div[3]/section/div[2]/div/div/div/div/div[3]/div[2]/button')
boton_sendcode.click()

sec_code=input()

for i in range(1,7):
    input_code= driver.find_element(by=By.XPATH,value='//*[@id="app"]/div[3]/section/div[2]/div/div/div/div/div[3]/div/div/div['+str(i)+']/input')
    input_code.send_keys(sec_code[i-1])

confirm_code = driver.find_element(by=By.XPATH,value='//*[@id="app"]/div[3]/section/div[2]/div/div/div/div/div[4]/div[2]/button')
confirm_code.click()

 419868


A esta altura ya estamos logueados en el CRM en la página /angecy_dashboard








In [120]:
#PREPARAMOS UNA FUNCIÓN PARA SELECCIONAR TODAS LAS COLUMNAS PARA QUE SE INCLUYAN EN LA EXPORTACION

def select_all_columns(driver):
    driver.find_element(by=By.XPATH,value='//*[@id="colViewButton"]').click()
    try:
        driver.find_element(by=By.XPATH,value='//*[@id="colViewButton"]').click()
    except:
        next
    columns=driver.find_elements(by=By.XPATH,value='/html/body/div[1]/div[3]/section/div/section/div[2]/div[2]/div[1]/ul//li')
    for i in range(len(columns)-6):
        driver.find_element(by=By.XPATH,value='/html/body/div[1]/div[3]/section/div/section/div[2]/div[2]/div[1]/ul/li['+str(len(columns))+']/div/input').click()
        time.sleep(0.2)

In [134]:
#FUNCION PARA DESCARGAR LA SMAT LIST YA ESTANDO FILTRADA Y CON LAS COLUMNAS SELECCIONADAS


def desc(driver):
    driver.find_element(by=By.XPATH,value='//*[@id="all"]').click()    # seleccionar la lista
    time.sleep(3)

    try:    
        driver.find_element(by=By.XPATH,value='//*[@id="hl_smartlists-main"]/div[1]/div/div[1]/div/span[2]').click()    
    except:
        next
        

    boton= WebDriverWait(driver,10).until(EC.presence_of_element_located((By.XPATH,'//*[@id="smartlists"]/div[2]/div[1]/span[10]/button')))
    boton.click()    #boton para exportar ya estaado en la smartlist
    
    boton= WebDriverWait(driver,10).until(EC.presence_of_element_located((By.XPATH,'/html/body/div[7]/div[1]/div/div/div/div[2]/div/div/div/button')))
    boton.click()    #boton para iniciar la descarga estando todo preparado
    
    time.sleep(1)
    WebDriverWait(driver,20).until(EC.element_to_be_clickable((By.XPATH,'//*[@id="all"]')))   


In [145]:
#Armo la lista de los Ids de las cuentas para poder navegar entre todas ellas y realizar las descargas

ALLS="6ZNDYU7FC7suAYtH81sC,vy3HXxVjRbBen4gDHTBT,335tvBVvpfEsc95omLWD"  #pego los Ids de las cuentas con las que queiro trabajar

ids=ALLS.split(",")   #armo la lista en base los Ids que pegué

In [146]:
# EJECUTO SECUENCIA PARA DESCARGAR TODOS LOS CONTACTS CON TODAS LAS COLUMNAS
#solo necesito estar logueado Y tener la lista de los ids de las cuentas que quiero descargar


for i in ids:
    url="https://CRMNAME.org/v2/location/"+i+"/contacts/smart_list/All"
    driver.get(url)
    time.sleep(10)
    select_all_columns(driver)  #preparo la lista
    desc(driver)  #descargo las cuentas
    print(i)  #voy imprimiendo los Ids de las cuentas que pude descargar


6ZNDYU7FC7suAYtH81sC
vy3HXxVjRbBen4gDHTBT
335tvBVvpfEsc95omLWD
